In [1]:
options(warn=-1)
suppressMessages({library("dplyr")
library("tidyr")
library("ggplot2")
library("GenomicRanges")
library("readxl")
library("data.table")
library("rtracklayer")
library("PRROC")})

# Combining and tydying CRISPRi data

In [2]:
overlapPairsWithLoops=function(loop_file, shift=0){
    loops <- read.csv(loop_file, header = F, sep = "\t")
    loop_vp_ranges <- GRanges(seqnames = loops$V1, ranges=IRanges(start=loops$V2, end=loops$V3))
    loop_OE_ranges <- GRanges(seqnames = loops$V4, ranges=IRanges(start=loops$V5-shift, end=loops$V6+shift))
    output <- CRISPRi$id %in% as.data.frame(findOverlaps(CRISPRi_OE_ranges, loop_OE_ranges))[paste(as.data.frame(findOverlaps(CRISPRi_OE_ranges, loop_OE_ranges))$queryHits,
        as.data.frame(findOverlaps(CRISPRi_OE_ranges, loop_OE_ranges))$subjectHits) %in% paste(as.data.frame(findOverlaps(CRISPRi_TSS_ranges, loop_vp_ranges))$queryHits, 
        as.data.frame(findOverlaps(CRISPRi_TSS_ranges, loop_vp_ranges))$subjectHits),]$queryHits
    return(output)}
overlapPairsWithLoopsSym=function(loop_file, shift=2500){
    loops <- read.csv(loop_file, header = F, sep = "\t")
    loop_bin1_ranges <- GRanges(seqnames = loops$V1, ranges=IRanges(start=loops$V2-shift, end=loops$V3+shift))
    loop_bin2_ranges <- GRanges(seqnames = loops$V4, ranges=IRanges(start=loops$V5-shift, end=loops$V6+shift))
    output <- CRISPRi$id %in% unique(as.data.frame(findOverlaps(CRISPRi_OE_ranges, loop_bin2_ranges))[paste(as.data.frame(findOverlaps(CRISPRi_OE_ranges, loop_bin2_ranges))$queryHits,
        as.data.frame(findOverlaps(CRISPRi_OE_ranges, loop_bin2_ranges))$subjectHits) %in% paste(as.data.frame(findOverlaps(CRISPRi_TSS_ranges, loop_bin1_ranges))$queryHits, 
        as.data.frame(findOverlaps(CRISPRi_TSS_ranges, loop_bin1_ranges))$subjectHits),]$queryHits) | 
        CRISPRi$id %in% unique(as.data.frame(findOverlaps(CRISPRi_OE_ranges, loop_bin1_ranges))[paste(as.data.frame(findOverlaps(CRISPRi_OE_ranges, loop_bin1_ranges))$queryHits, 
        as.data.frame(findOverlaps(CRISPRi_OE_ranges, loop_bin1_ranges))$subjectHits) %in% paste(as.data.frame(findOverlaps(CRISPRi_TSS_ranges, loop_bin2_ranges))$queryHits, 
        as.data.frame(findOverlaps(CRISPRi_TSS_ranges, loop_bin2_ranges))$subjectHits),]$queryHits)
    return(output)}

gasperini_neg <- read.csv("Auxiliary_data/CRISPRi_data/GSE120861_all_deg_results.at_scale.txt.gz", 
                          header = T, sep = "\t")
genes <- unique(gasperini_neg[,c(9,16)])
genes <- filter(genes, !(duplicated(genes$gene_short_name)))
gasperini_neg <- filter(gasperini_neg, quality_rank_grna=="top_two" & !(outlier_gene) &
                        fold_change.transcript_remaining<1.1 &
                        (pvalue.empirical.adjusted > 0.1|pvalue.empirical.adjusted=="not_applicable"))
gasperini_neg <- gasperini_neg[,c(12:14,9,16)]
gasperini_neg$Significant <- FALSE
gasperini_neg[,2] <- as.integer(gasperini_neg[,2])
gasperini_neg[,3] <- as.integer(gasperini_neg[,3])
gasperini_pos <- read_excel("Auxiliary_data/CRISPRi_data/Gasperini2019_TableS2.xlsx", sheet = 3)
gasperini_pos <- filter(gasperini_pos, high_confidence_subset)
gasperini_pos <- gasperini_pos[,c(9:11,3)]
colnames(gasperini_pos) <- colnames(gasperini_neg)[1:4]
gasperini_pos <- left_join(gasperini_pos,genes)
gasperini_pos$Significant <- TRUE
fulco <- read_excel("Auxiliary_data/CRISPRi_data/Fulco2019_SupTables.xlsx", sheet = 16, skip = 1)
fulco <- filter(fulco, class != "promoter" & 
                ((`Fraction change in gene expr`< 0 & Significant == T)| (Significant == F)))
fulco <- fulco[,c(1,2,3,5,18,10)]
colnames(fulco) <- colnames(gasperini_neg)
CRISPRi <- bind_rows(gasperini_pos, gasperini_neg,fulco)
CRISPRi<-filter(CRISPRi,abs((CRISPRi$target_site.start+CRISPRi$target_site.stop)/2-CRISPRi$target_gene.start)<1000000&
                abs((CRISPRi$target_site.start+CRISPRi$target_site.stop)/2-CRISPRi$target_gene.start)>5000)
CRISPRi_TSS_ranges <- GRanges(seqnames = CRISPRi$target_site.chr, 
                              ranges=IRanges(start=CRISPRi$target_gene.start, end=CRISPRi$target_gene.start+1))
vp <- read.csv("MChIPC_output/MChIPC_viewpoints.bed", header=F, sep="\t") 
# here I should later add coverage filter
vp_ranges <- GRanges(seqnames = vp$V1, ranges=IRanges(start=vp$V2, end=vp$V3))
CRISPRi <- CRISPRi[as.data.frame(findOverlaps(CRISPRi_TSS_ranges, vp_ranges))$queryHits,]
CRISPRi_TSS_ranges <- GRanges(seqnames = CRISPRi$target_site.chr, 
                             ranges=IRanges(start=CRISPRi$target_gene.start, end=CRISPRi$target_gene.start+1))
CRISPRi_OE_ranges <- GRanges(seqnames = CRISPRi$target_site.chr, 
                             ranges=IRanges(start=CRISPRi$target_site.start, end=CRISPRi$target_site.stop))
CRISPRi$id <- seq(1,nrow(CRISPRi))
CRISPRi$MChIPC.loop <- overlapPairsWithLoops("MChIPC_output/MChIPC_interactions.bedpe", shift=500)
CRISPRi$PLACseq.loop <- overlapPairsWithLoops("PLACseq_output/PLACseq_interactions.bedpe")
CRISPRi$MicroC.loop <- overlapPairsWithLoopsSym("MicroC_output/MicroC_interactions.bedpe", shift=2500)
CRISPRi$HiC.loop <- overlapPairsWithLoopsSym("HiC_output/HiC_interactions.bedpe", shift=2500)
CRISPRi$MChIPC.sub05.loop <- overlapPairsWithLoops("MChIPC_output/MChIPC_interactions.sub_0.5.bedpe", shift=500)
CRISPRi$MChIPC.sub025.loop <- overlapPairsWithLoops("MChIPC_output/MChIPC_interactions.sub_0.25.bedpe", shift=500)
CRISPRi$MChIPC.sub01.loop <- overlapPairsWithLoops("MChIPC_output/MChIPC_interactions.sub_0.1.bedpe", shift=500)
CTCF_peaks <- read.csv("Auxiliary_data/ENCFF002CEL.bed.gz", header = F, sep = "\t")
CTCF_peaks <- filter(CTCF_peaks, V5 > 250)
CTCF_ranges <- GRanges(seqnames = CTCF_peaks$V1, 
                       ranges=IRanges(start=CTCF_peaks$V2-1000, end=CTCF_peaks$V3+1000))
CRISPRi$overlaps.CTCF <- CRISPRi$id %in% as.data.frame(findOverlaps(CRISPRi_OE_ranges, CTCF_ranges))$queryHits
CRISPRi$id <- NULL
CRISPRi_pos <- filter(CRISPRi, Significant)
CRISPRi_neg <- filter(CRISPRi, !(Significant))
# some basic statistics:
cat(paste0("Overall we analyzed ",nrow(CRISPRi_pos)," CRISPRi-verified E-P pairs in K562.\n"))
cat(paste0(nrow(filter(CRISPRi_pos,MChIPC.loop))," (",round(100*nrow(filter(CRISPRi_pos,MChIPC.loop))/nrow(CRISPRi_pos),1),
"%) of them had an underlying (physical) contact in MChIP-C data,\n"))
cat(paste0(nrow(filter(CRISPRi_pos, MicroC.loop))," (",round(100*nrow(filter(CRISPRi_pos,MicroC.loop))/nrow(CRISPRi_pos),1),
"%) - in Micro-C data,\n"))
cat(paste0(nrow(filter(CRISPRi_pos, HiC.loop))," (",round(100*nrow(filter(CRISPRi_pos,HiC.loop))/nrow(CRISPRi_pos),1),
"%) - in Hi-C data.\n"))
#cat(paste0("When we considered only ",nrow(filter(CRISPRi_pos, coverage>5000)),
#" E-P pairs with high-coverage viewpoints we found out that even higher proportion (",
#100*nrow(filter(CRISPRi_pos, coverage>5000&MChIPC.loop))/nrow(filter(CRISPRi_pos, coverage>5000)),
#"%) of them had underlying (physical) contact in MChIP-C data."))
      
cat(paste0("As a control we also analyzed physical contacts within ",nrow(CRISPRi_neg)," non-functional DHS-P pairs.\n"))
cat(paste0(nrow(filter(CRISPRi_neg,MChIPC.loop))," (",round(100*nrow(filter(CRISPRi_neg,MChIPC.loop))/nrow(CRISPRi_neg),1), 
"%) of them had an underlying (physical) contact in MChIP-C data,\n"))
cat(paste0(nrow(filter(CRISPRi_neg, MicroC.loop))," (",round(100*nrow(filter(CRISPRi_neg,MicroC.loop))/nrow(CRISPRi_neg),1),
"%) - in Micro-C data,\n"))
cat(paste0(nrow(filter(CRISPRi_neg, HiC.loop))," (",round(100*nrow(filter(CRISPRi_neg,HiC.loop))/nrow(CRISPRi_neg),1),
"%) - in Hi-C data.\n"))

N_PLAC_pos <- nrow(filter(CRISPRi_pos,abs((CRISPRi_pos$target_site.start+CRISPRi_pos$target_site.stop)/2-CRISPRi_pos$target_gene.start)>10000))
N_PLAC_neg <- nrow(filter(CRISPRi_neg,abs((CRISPRi_neg$target_site.start+CRISPRi_neg$target_site.stop)/2-CRISPRi_neg$target_gene.start)>10000))
cat(paste0("For PLAC-seq we were able to analyze only ",N_PLAC_pos," verified E-P pairs and ", N_PLAC_neg,
" non-functional DHS-P pairs\ndue to the lower resolution of PLAC-seq interaction calling data.\n"))            
cat(paste0(nrow(filter(CRISPRi_pos, PLACseq.loop))," (",round(100*nrow(filter(CRISPRi_pos,PLACseq.loop))/N_PLAC_pos,1), 
"%) of analyzed E-P pairs and ",
nrow(filter(CRISPRi_neg, PLACseq.loop))," (",round(100*nrow(filter(CRISPRi_neg,PLACseq.loop))/N_PLAC_neg,1),
"%) of analyzed non-functional DHS-P pairs\nhad an underlying (physical) PLAC-seq contact.\n"))
cat("We also calculated precision metrics for E-P predictors based on studied proximity-ligation methods.\n")
cat(paste0("Precisions were estimated to be equal to: ", 
           round(100*nrow(filter(CRISPRi_pos, MChIPC.loop))/nrow(filter(CRISPRi, MChIPC.loop)),1),"% (MChIP-C),",
           round(100*nrow(filter(CRISPRi_pos, PLACseq.loop))/nrow(filter(CRISPRi, PLACseq.loop)),1),"% (PLAC-seq),",
           round(100*nrow(filter(CRISPRi_pos, MicroC.loop))/nrow(filter(CRISPRi, MicroC.loop)),1),"% (Micro-C) and ",
           round(100*nrow(filter(CRISPRi_pos, HiC.loop))/nrow(filter(CRISPRi, HiC.loop)),1),"% (Hi-C).\n"))
cat(paste0("Finally we analyzed 50%-, 25%- and 10%-subsampled MChIP-C datasets. Their sensitivity was (correspondingly): ",
          round(100*nrow(filter(CRISPRi_pos,MChIPC.sub05.loop))/nrow(CRISPRi_pos),1), "%, ",
          round(100*nrow(filter(CRISPRi_pos,MChIPC.sub025.loop))/nrow(CRISPRi_pos),1), "%, and ",
          round(100*nrow(filter(CRISPRi_pos,MChIPC.sub01.loop))/nrow(CRISPRi_pos),1), "%. False positive rate: ",
          round(100*nrow(filter(CRISPRi_neg,MChIPC.sub05.loop))/nrow(CRISPRi_neg),1), "%, ",
          round(100*nrow(filter(CRISPRi_neg,MChIPC.sub025.loop))/nrow(CRISPRi_neg),1), "%, and ",
          round(100*nrow(filter(CRISPRi_neg,MChIPC.sub01.loop))/nrow(CRISPRi_neg),1), "%. Precision: ",
          round(100*nrow(filter(CRISPRi_pos, MChIPC.sub05.loop))/nrow(filter(CRISPRi, MChIPC.sub05.loop)),1), "%, ",
          round(100*nrow(filter(CRISPRi_pos, MChIPC.sub025.loop))/nrow(filter(CRISPRi, MChIPC.sub025.loop)),1), "%, and ",
          round(100*nrow(filter(CRISPRi_pos, MChIPC.sub01.loop))/nrow(filter(CRISPRi, MChIPC.sub01.loop)),1), "%."))               
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
write.table(CRISPRi_pos, "Summary_output_datasets/E-P_pairs.txt", col.names=T, sep = "\t")
write.table(CRISPRi_neg, "Summary_output_datasets/DHS-P_pairs.txt", col.names=T, sep = "\t")

Joining, by = "gene_short_name"


Overall we analyzed 362 CRISPRi-verified E-P pairs in K562.
219 (60.5%) of them had an underlying (physical) contact in MChIP-C data,
70 (19.3%) - in Micro-C data,
7 (1.9%) - in Hi-C data.
As a control we also analyzed physical contacts within 54930 non-functional DHS-P pairs.
3521 (6.4%) of them had an underlying (physical) contact in MChIP-C data,
915 (1.7%) - in Micro-C data,
321 (0.6%) - in Hi-C data.
For PLAC-seq we were able to analyze only 311 verified E-P pairs and 54600 non-functional DHS-P pairs
due to the lower resolution of PLAC-seq interaction calling data.
43 (13.8%) of analyzed E-P pairs and 1031 (1.9%) of analyzed non-functional DHS-P pairs
had an underlying (physical) PLAC-seq contact.
We also calculated precision metrics for E-P predictors based on studied proximity-ligation methods.
Precisions were estimated to be equal to: 5.9% (MChIP-C),4% (PLAC-seq),7.1% (Micro-C) and 2.1% (Hi-C).
Finally we analyzed 50%-, 25%- and 10%-subsampled MChIP-C datasets. Their sensitivit

# Plotting E(DHS)-P interaction heatmaps and average profiles

# E-P pairs physically linked in MChIP-C dataset

In [ ]:
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_pos$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_pos$target_gene.start, end=CRISPRi_pos$target_gene.start+1))
MChIPC_signal <- read.csv("MChIPC_output/MChIPC_interaction_data.txt", header = T, sep = "\t")
MChIPC_signal <- filter(MChIPC_signal, dist_rank!=0)
signal_ranges <- GRanges(seqnames = MChIPC_signal$vp_chr, 
                         ranges=IRanges(start=MChIPC_signal$vp_start, end=MChIPC_signal$vp_end))
MChIPC_signal <- MChIPC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
MChIPC_signal <- filter(MChIPC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = MChIPC_signal$vp_chr, 
                         ranges=IRanges(start=MChIPC_signal$vp_start, end=MChIPC_signal$vp_end))
CRISPRi_pos_ovrl <- filter(CRISPRi_pos, MChIPC.loop==TRUE & overlaps.CTCF==FALSE)
CRISPRi_pos_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_pos_ovrl))){
  loop <- CRISPRi_pos_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- MChIPC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal <- loop_signal[,c(21,28)]
  colnames(loop_signal)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_pos_ovrl_heatmap <- left_join(CRISPRi_pos_ovrl_heatmap, loop_signal)})}
CRISPRi_pos_ovrl_heatmap[is.na(CRISPRi_pos_ovrl_heatmap)] <- 0
CRISPRi_pos_ovrl_heatmap <- as.data.frame(t(CRISPRi_pos_ovrl_heatmap)[2:ncol(CRISPRi_pos_ovrl_heatmap),])
CRISPRi_pos_ovrl_heatmap <- arrange(CRISPRi_pos_ovrl_heatmap, rowMeans(CRISPRi_pos_ovrl_heatmap))
CRISPRi_pos_ovrl_heatmap_mod <- CRISPRi_pos_ovrl_heatmap
CRISPRi_pos_ovrl_heatmap_mod[CRISPRi_pos_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.4/4a_int_EP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_pos_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_pos_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("MChIP-C signal")

In [ ]:
ggsave("Figures/Fig.4/4a_int_EP_profile.pdf",device="pdf")

# E-P pairs with no physical link in MChIP-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_pos$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_pos$target_gene.start, end=CRISPRi_pos$target_gene.start+1))
MChIPC_signal <- read.csv("MChIPC_output/MChIPC_interaction_data.txt", header = T, sep = "\t")
MChIPC_signal <- filter(MChIPC_signal, dist_rank!=0)
signal_ranges <- GRanges(seqnames = MChIPC_signal$vp_chr, 
                         ranges=IRanges(start=MChIPC_signal$vp_start, end=MChIPC_signal$vp_end))
MChIPC_signal <- MChIPC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
MChIPC_signal <- filter(MChIPC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = MChIPC_signal$vp_chr, 
                         ranges=IRanges(start=MChIPC_signal$vp_start, end=MChIPC_signal$vp_end))
CRISPRi_pos_non_ovrl <- filter(CRISPRi_pos, MChIPC.loop==FALSE & overlaps.CTCF==FALSE)
CRISPRi_pos_non_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_pos_non_ovrl))){
  loop <- CRISPRi_pos_non_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- MChIPC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal <- loop_signal[,c(21,28)]
  colnames(loop_signal)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_pos_non_ovrl_heatmap <- left_join(CRISPRi_pos_non_ovrl_heatmap, loop_signal)})}
CRISPRi_pos_non_ovrl_heatmap[is.na(CRISPRi_pos_non_ovrl_heatmap)] <- 0
CRISPRi_pos_non_ovrl_heatmap <- as.data.frame(t(CRISPRi_pos_non_ovrl_heatmap)[2:ncol(CRISPRi_pos_non_ovrl_heatmap),])
CRISPRi_pos_non_ovrl_heatmap <- arrange(CRISPRi_pos_non_ovrl_heatmap, rowMeans(CRISPRi_pos_non_ovrl_heatmap))
CRISPRi_pos_non_ovrl_heatmap_mod <- CRISPRi_pos_non_ovrl_heatmap
CRISPRi_pos_non_ovrl_heatmap_mod[CRISPRi_pos_non_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.4/4a_none_EP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_pos_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_pos_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_non_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("MChIP-C signal")

In [ ]:
ggsave("Figures/Fig.4/4a_none_EP_profile.pdf",device="pdf")

# DHS-P pairs physically linked in MChIP-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_neg$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_neg$target_gene.start, end=CRISPRi_neg$target_gene.start+1))
MChIPC_signal <- read.csv("MChIPC_output/MChIPC_interaction_data.txt", header = T, sep = "\t")
MChIPC_signal <- filter(MChIPC_signal, dist_rank!=0)
signal_ranges <- GRanges(seqnames = MChIPC_signal$vp_chr, 
                         ranges=IRanges(start=MChIPC_signal$vp_start, end=MChIPC_signal$vp_end))
MChIPC_signal <- MChIPC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
MChIPC_signal <- filter(MChIPC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = MChIPC_signal$vp_chr, 
                         ranges=IRanges(start=MChIPC_signal$vp_start, end=MChIPC_signal$vp_end))
CRISPRi_neg_ovrl <- filter(CRISPRi_neg, MChIPC.loop==TRUE & overlaps.CTCF==FALSE)
CRISPRi_neg_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_neg_ovrl))){
  loop <- CRISPRi_neg_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- MChIPC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal <- loop_signal[,c(21,28)]
  colnames(loop_signal)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_neg_ovrl_heatmap <- left_join(CRISPRi_neg_ovrl_heatmap, loop_signal)})}
CRISPRi_neg_ovrl_heatmap[is.na(CRISPRi_neg_ovrl_heatmap)] <- 0
CRISPRi_neg_ovrl_heatmap <- as.data.frame(t(CRISPRi_neg_ovrl_heatmap)[2:ncol(CRISPRi_neg_ovrl_heatmap),])
CRISPRi_neg_ovrl_heatmap <- arrange(CRISPRi_neg_ovrl_heatmap, rowMeans(CRISPRi_neg_ovrl_heatmap))
CRISPRi_neg_ovrl_heatmap_mod <- CRISPRi_neg_ovrl_heatmap
CRISPRi_neg_ovrl_heatmap_mod[CRISPRi_neg_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.4/4a_int_DHSP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_neg_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_neg_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from DHS (250bp bins)")+ylab("MChIP-C signal")

In [ ]:
ggsave("Figures/Fig.4/4a_int_DHSP_profile.pdf",device="pdf")

# DHS-P pairs with no physical link in MChIPC dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_neg$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_neg$target_gene.start, end=CRISPRi_neg$target_gene.start+1))
MChIPC_signal <- read.csv("MChIPC_output/MChIPC_interaction_data.txt", header = T, sep = "\t")
MChIPC_signal <- filter(MChIPC_signal, dist_rank!=0)
signal_ranges <- GRanges(seqnames = MChIPC_signal$vp_chr, 
                         ranges=IRanges(start=MChIPC_signal$vp_start, end=MChIPC_signal$vp_end))
MChIPC_signal <- MChIPC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
MChIPC_signal <- filter(MChIPC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = MChIPC_signal$vp_chr, 
                         ranges=IRanges(start=MChIPC_signal$vp_start, end=MChIPC_signal$vp_end))
CRISPRi_neg_non_ovrl <- filter(CRISPRi_neg, MChIPC.loop==FALSE & overlaps.CTCF==FALSE)
CRISPRi_neg_non_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_neg_non_ovrl))){
  loop <- CRISPRi_neg_non_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- MChIPC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal <- loop_signal[,c(21,28)]
  colnames(loop_signal)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_neg_non_ovrl_heatmap <- left_join(CRISPRi_neg_non_ovrl_heatmap, loop_signal)})}
CRISPRi_neg_non_ovrl_heatmap[is.na(CRISPRi_neg_non_ovrl_heatmap)] <- 0
CRISPRi_neg_non_ovrl_heatmap <- as.data.frame(t(CRISPRi_neg_non_ovrl_heatmap)[2:ncol(CRISPRi_neg_non_ovrl_heatmap),])
CRISPRi_neg_non_ovrl_heatmap <- arrange(CRISPRi_neg_non_ovrl_heatmap, rowMeans(CRISPRi_neg_non_ovrl_heatmap))
CRISPRi_neg_non_ovrl_heatmap_mod <- CRISPRi_neg_non_ovrl_heatmap
CRISPRi_neg_non_ovrl_heatmap_mod[CRISPRi_neg_non_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.4/4a_none_DHSP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_neg_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_neg_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_non_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from DHS (250bp bins)")+ylab("MChIP-C signal")

In [ ]:
ggsave("Figures/Fig.4/4a_none_DHSP_profile.pdf",device="pdf")

# E-P pairs physically linked in PLAC-seq dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_pos$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_pos$target_gene.start, end=CRISPRi_pos$target_gene.start+1))
PLACseq_signal <- read.csv("PLACseq_output/PLACseq.signal.df.txt", header = F, sep = "\t")
PLACseq_signal$V7 <- as.numeric(gsub(",",".",PLACseq_signal$V7))
colnames(PLACseq_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum")
signal_ranges <- GRanges(seqnames = PLACseq_signal$vp_chr, 
                         ranges=IRanges(start=PLACseq_signal$vp_start, end=PLACseq_signal$vp_end))
PLACseq_signal <- PLACseq_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
PLACseq_signal$dist_rank <- 0
PLACseq_signal$dist_rank[PLACseq_signal$vp_start>PLACseq_signal$OE_start]<-(PLACseq_signal$vp_start[PLACseq_signal$vp_start>PLACseq_signal$OE_start]- 
                                                            PLACseq_signal$OE_start[PLACseq_signal$vp_start>PLACseq_signal$OE_start])/250
PLACseq_signal$dist_rank[PLACseq_signal$vp_end<PLACseq_signal$OE_start]<-(PLACseq_signal$OE_end[PLACseq_signal$vp_end<PLACseq_signal$OE_start]- 
                                                            PLACseq_signal$vp_end[PLACseq_signal$vp_end<PLACseq_signal$OE_start])/250
PLACseq_signal <- filter(PLACseq_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = PLACseq_signal$vp_chr, 
                         ranges=IRanges(start=PLACseq_signal$vp_start, end=PLACseq_signal$vp_end))
CRISPRi_pos_ovrl <- filter(CRISPRi_pos, PLACseq.loop==TRUE & overlaps.CTCF==FALSE &
                          abs((target_site.start+target_site.stop)/2-target_gene.start)>10000)
CRISPRi_pos_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_pos_ovrl))){
  loop <- CRISPRi_pos_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- PLACseq_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal <- loop_signal[,c(8,7)]
  colnames(loop_signal)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_pos_ovrl_heatmap <- left_join(CRISPRi_pos_ovrl_heatmap, loop_signal)})}
CRISPRi_pos_ovrl_heatmap[is.na(CRISPRi_pos_ovrl_heatmap)] <- 0
CRISPRi_pos_ovrl_heatmap <- as.data.frame(t(CRISPRi_pos_ovrl_heatmap)[2:ncol(CRISPRi_pos_ovrl_heatmap),])
CRISPRi_pos_ovrl_heatmap <- arrange(CRISPRi_pos_ovrl_heatmap, rowMeans(CRISPRi_pos_ovrl_heatmap))
CRISPRi_pos_ovrl_heatmap_mod <- CRISPRi_pos_ovrl_heatmap
CRISPRi_pos_ovrl_heatmap_mod[CRISPRi_pos_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_PLACseq_int_EP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_pos_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_pos_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("PLAC-seq signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_PLACseq_int_EP_profile.pdf",device="pdf")

# E-P pairs with no physical link in PLAC-seq dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_pos$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_pos$target_gene.start, end=CRISPRi_pos$target_gene.start+1))
PLACseq_signal <- read.csv("PLACseq_output/PLACseq.signal.df.txt", header = F, sep = "\t")
PLACseq_signal$V7 <- as.numeric(gsub(",",".",PLACseq_signal$V7))
colnames(PLACseq_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum")
signal_ranges <- GRanges(seqnames = PLACseq_signal$vp_chr, 
                         ranges=IRanges(start=PLACseq_signal$vp_start, end=PLACseq_signal$vp_end))
PLACseq_signal <- PLACseq_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
PLACseq_signal$dist_rank <- 0
PLACseq_signal$dist_rank[PLACseq_signal$vp_start>PLACseq_signal$OE_start]<-(PLACseq_signal$vp_start[PLACseq_signal$vp_start>PLACseq_signal$OE_start]- 
                                                            PLACseq_signal$OE_start[PLACseq_signal$vp_start>PLACseq_signal$OE_start])/250
PLACseq_signal$dist_rank[PLACseq_signal$vp_end<PLACseq_signal$OE_start]<-(PLACseq_signal$OE_end[PLACseq_signal$vp_end<PLACseq_signal$OE_start]- 
                                                            PLACseq_signal$vp_end[PLACseq_signal$vp_end<PLACseq_signal$OE_start])/250
PLACseq_signal <- filter(PLACseq_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = PLACseq_signal$vp_chr, 
                         ranges=IRanges(start=PLACseq_signal$vp_start, end=PLACseq_signal$vp_end))
CRISPRi_pos_non_ovrl <- filter(CRISPRi_pos, PLACseq.loop==FALSE & overlaps.CTCF==FALSE &
                              abs((target_site.start+target_site.stop)/2-target_gene.start)>10000)
CRISPRi_pos_non_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_pos_non_ovrl))){
  loop <- CRISPRi_pos_non_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- PLACseq_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal <- loop_signal[,c(8,7)]
  colnames(loop_signal)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_pos_non_ovrl_heatmap <- left_join(CRISPRi_pos_non_ovrl_heatmap, loop_signal)})}
CRISPRi_pos_non_ovrl_heatmap[is.na(CRISPRi_pos_non_ovrl_heatmap)] <- 0
CRISPRi_pos_non_ovrl_heatmap <- as.data.frame(t(CRISPRi_pos_non_ovrl_heatmap)[2:ncol(CRISPRi_pos_non_ovrl_heatmap),])
CRISPRi_pos_non_ovrl_heatmap <- arrange(CRISPRi_pos_non_ovrl_heatmap, rowMeans(CRISPRi_pos_non_ovrl_heatmap))
CRISPRi_pos_non_ovrl_heatmap_mod <- CRISPRi_pos_non_ovrl_heatmap
CRISPRi_pos_non_ovrl_heatmap_mod[CRISPRi_pos_non_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_PLACseq_none_EP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_pos_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_pos_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_non_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("PLAC-seq signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_PLACseq_none_EP_profile.pdf",device="pdf")

# DHS-P pairs physically linked in PLAC-seq dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_neg$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_neg$target_gene.start, end=CRISPRi_neg$target_gene.start+1))
PLACseq_signal <- read.csv("PLACseq_output/PLACseq.signal.df.txt", header = F, sep = "\t")
PLACseq_signal$V7 <- as.numeric(gsub(",",".",PLACseq_signal$V7))
colnames(PLACseq_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum")
signal_ranges <- GRanges(seqnames = PLACseq_signal$vp_chr, 
                         ranges=IRanges(start=PLACseq_signal$vp_start, end=PLACseq_signal$vp_end))
PLACseq_signal <- PLACseq_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
PLACseq_signal$dist_rank <- 0
PLACseq_signal$dist_rank[PLACseq_signal$vp_start>PLACseq_signal$OE_start]<-(PLACseq_signal$vp_start[PLACseq_signal$vp_start>PLACseq_signal$OE_start]- 
                                                            PLACseq_signal$OE_start[PLACseq_signal$vp_start>PLACseq_signal$OE_start])/250
PLACseq_signal$dist_rank[PLACseq_signal$vp_end<PLACseq_signal$OE_start]<-(PLACseq_signal$OE_end[PLACseq_signal$vp_end<PLACseq_signal$OE_start]- 
                                                            PLACseq_signal$vp_end[PLACseq_signal$vp_end<PLACseq_signal$OE_start])/250
PLACseq_signal <- filter(PLACseq_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = PLACseq_signal$vp_chr, 
                         ranges=IRanges(start=PLACseq_signal$vp_start, end=PLACseq_signal$vp_end))
CRISPRi_neg_ovrl <- filter(CRISPRi_neg, PLACseq.loop==TRUE & overlaps.CTCF==FALSE &
                          abs((target_site.start+target_site.stop)/2-target_gene.start)>10000)
CRISPRi_neg_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_neg_ovrl))){
  loop <- CRISPRi_neg_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- PLACseq_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal <- loop_signal[,c(8,7)]
  colnames(loop_signal)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_neg_ovrl_heatmap <- left_join(CRISPRi_neg_ovrl_heatmap, loop_signal)})}
CRISPRi_neg_ovrl_heatmap[is.na(CRISPRi_neg_ovrl_heatmap)] <- 0
CRISPRi_neg_ovrl_heatmap <- as.data.frame(t(CRISPRi_neg_ovrl_heatmap)[2:ncol(CRISPRi_neg_ovrl_heatmap),])
CRISPRi_neg_ovrl_heatmap <- arrange(CRISPRi_neg_ovrl_heatmap, rowMeans(CRISPRi_neg_ovrl_heatmap))
CRISPRi_neg_ovrl_heatmap_mod <- CRISPRi_neg_ovrl_heatmap
CRISPRi_neg_ovrl_heatmap_mod[CRISPRi_neg_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_PLACseq_int_DHSP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_neg_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_neg_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from DHS (250bp bins)")+ylab("PLAC-seq signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_PLACseq_int_DHSP_profile.pdf",device="pdf")

# DHS-P pairs with no physical link in PLAC-seq dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_neg$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_neg$target_gene.start, end=CRISPRi_neg$target_gene.start+1))
PLACseq_signal <- read.csv("PLACseq_output/PLACseq.signal.df.txt", header = F, sep = "\t")
PLACseq_signal$V7 <- as.numeric(gsub(",",".",PLACseq_signal$V7))
colnames(PLACseq_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum")
signal_ranges <- GRanges(seqnames = PLACseq_signal$vp_chr, 
                         ranges=IRanges(start=PLACseq_signal$vp_start, end=PLACseq_signal$vp_end))
PLACseq_signal <- PLACseq_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
PLACseq_signal$dist_rank <- 0
PLACseq_signal$dist_rank[PLACseq_signal$vp_start>PLACseq_signal$OE_start]<-(PLACseq_signal$vp_start[PLACseq_signal$vp_start>PLACseq_signal$OE_start]- 
                                                            PLACseq_signal$OE_start[PLACseq_signal$vp_start>PLACseq_signal$OE_start])/250
PLACseq_signal$dist_rank[PLACseq_signal$vp_end<PLACseq_signal$OE_start]<-(PLACseq_signal$OE_end[PLACseq_signal$vp_end<PLACseq_signal$OE_start]- 
                                                            PLACseq_signal$vp_end[PLACseq_signal$vp_end<PLACseq_signal$OE_start])/250
PLACseq_signal <- filter(PLACseq_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = PLACseq_signal$vp_chr, 
                         ranges=IRanges(start=PLACseq_signal$vp_start, end=PLACseq_signal$vp_end))
CRISPRi_neg_non_ovrl <- filter(CRISPRi_neg, PLACseq.loop==FALSE & overlaps.CTCF==FALSE &
                              abs((target_site.start+target_site.stop)/2-target_gene.start)>10000)
CRISPRi_neg_non_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_neg_non_ovrl))){
  loop <- CRISPRi_neg_non_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- PLACseq_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal <- loop_signal[,c(8,7)]
  colnames(loop_signal)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_neg_non_ovrl_heatmap <- left_join(CRISPRi_neg_non_ovrl_heatmap, loop_signal)})}
CRISPRi_neg_non_ovrl_heatmap[is.na(CRISPRi_neg_non_ovrl_heatmap)] <- 0
CRISPRi_neg_non_ovrl_heatmap <- as.data.frame(t(CRISPRi_neg_non_ovrl_heatmap)[2:ncol(CRISPRi_neg_non_ovrl_heatmap),])
CRISPRi_neg_non_ovrl_heatmap <- arrange(CRISPRi_neg_non_ovrl_heatmap, rowMeans(CRISPRi_neg_non_ovrl_heatmap))
CRISPRi_neg_non_ovrl_heatmap_mod <- CRISPRi_neg_non_ovrl_heatmap
CRISPRi_neg_non_ovrl_heatmap_mod[CRISPRi_neg_non_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_PLACseq_none_DHSP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_neg_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_neg_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_non_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from DHS (250bp bins)")+ylab("PLAC-seq signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_PLACseq_none_DHSP_profile.pdf",device="pdf")

# E-P pairs physically linked in Micro-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_pos$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_pos$target_gene.start, end=CRISPRi_pos$target_gene.start+1))
MicroC_signal <- read.csv("MicroC_output/MicroC.signal.df.txt", header = F, sep = "\t") %>% filter(V1!="")
colnames(MicroC_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum", "N_sum_ICE")
signal_ranges <- GRanges(seqnames = MicroC_signal$vp_chr, 
                         ranges=IRanges(start=MicroC_signal$vp_start, end=MicroC_signal$vp_end))
MicroC_signal <- MicroC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
MicroC_signal$dist_rank <- 0
MicroC_signal$dist_rank[MicroC_signal$vp_start>MicroC_signal$OE_start]<-(MicroC_signal$vp_start[MicroC_signal$vp_start>MicroC_signal$OE_start]- 
                                                            MicroC_signal$OE_start[MicroC_signal$vp_start>MicroC_signal$OE_start])/250
MicroC_signal$dist_rank[MicroC_signal$vp_end<MicroC_signal$OE_start]<-(MicroC_signal$OE_end[MicroC_signal$vp_end<MicroC_signal$OE_start]- 
                                                            MicroC_signal$vp_end[MicroC_signal$vp_end<MicroC_signal$OE_start])/250
MicroC_signal <- filter(MicroC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = MicroC_signal$vp_chr, 
                         ranges=IRanges(start=MicroC_signal$vp_start, end=MicroC_signal$vp_end))
CRISPRi_pos_ovrl <- filter(CRISPRi_pos, MicroC.loop==TRUE & overlaps.CTCF==FALSE)
CRISPRi_pos_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
CRISPRi_pos_ovrl_heatmap_ICE <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_pos_ovrl))){
  loop <- CRISPRi_pos_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- MicroC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal_n <- loop_signal[,c(9,7)]
  loop_signal_ICE <- loop_signal[,c(9,8)]
  colnames(loop_signal_n)[2] <- paste0("loop_",i)
  colnames(loop_signal_ICE)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_pos_ovrl_heatmap <- left_join(CRISPRi_pos_ovrl_heatmap, loop_signal_n)})
  suppressMessages({CRISPRi_pos_ovrl_heatmap_ICE <- left_join(CRISPRi_pos_ovrl_heatmap_ICE, loop_signal_ICE)})}
CRISPRi_pos_ovrl_heatmap[is.na(CRISPRi_pos_ovrl_heatmap)] <- 0
CRISPRi_pos_ovrl_heatmap_ICE[is.na(CRISPRi_pos_ovrl_heatmap_ICE)] <- 0
CRISPRi_pos_ovrl_heatmap <- as.data.frame(t(CRISPRi_pos_ovrl_heatmap)[2:ncol(CRISPRi_pos_ovrl_heatmap),])
CRISPRi_pos_ovrl_heatmap_ICE <- as.data.frame(t(CRISPRi_pos_ovrl_heatmap_ICE)[2:ncol(CRISPRi_pos_ovrl_heatmap_ICE),])
CRISPRi_pos_ovrl_heatmap <- arrange(CRISPRi_pos_ovrl_heatmap, rowMeans(CRISPRi_pos_ovrl_heatmap))
CRISPRi_pos_ovrl_heatmap_ICE <- arrange(CRISPRi_pos_ovrl_heatmap_ICE, rowMeans(CRISPRi_pos_ovrl_heatmap_ICE))
CRISPRi_pos_ovrl_heatmap_mod <- CRISPRi_pos_ovrl_heatmap
CRISPRi_pos_ovrl_heatmap_mod[CRISPRi_pos_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_MicroC_int_EP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_pos_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_pos_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# raw profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Micro-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_MicroC_int_EP_profile.pdf",device="pdf")

In [ ]:
# ICE profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=1000*colMeans(CRISPRi_pos_ovrl_heatmap_ICE)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Micro-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_MicroC_int_EP_profile_ICE.pdf",device="pdf")

# E-P pairs with no physical link in Micro-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_pos$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_pos$target_gene.start, end=CRISPRi_pos$target_gene.start+1))
MicroC_signal <- read.csv("MicroC_output/MicroC.signal.df.txt", header = F, sep = "\t") %>% filter(V1!="")
colnames(MicroC_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum", "N_sum_ICE")
signal_ranges <- GRanges(seqnames = MicroC_signal$vp_chr, 
                         ranges=IRanges(start=MicroC_signal$vp_start, end=MicroC_signal$vp_end))
MicroC_signal <- MicroC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
MicroC_signal$dist_rank <- 0
MicroC_signal$dist_rank[MicroC_signal$vp_start>MicroC_signal$OE_start]<-(MicroC_signal$vp_start[MicroC_signal$vp_start>MicroC_signal$OE_start]- 
                                                            MicroC_signal$OE_start[MicroC_signal$vp_start>MicroC_signal$OE_start])/250
MicroC_signal$dist_rank[MicroC_signal$vp_end<MicroC_signal$OE_start]<-(MicroC_signal$OE_end[MicroC_signal$vp_end<MicroC_signal$OE_start]- 
                                                            MicroC_signal$vp_end[MicroC_signal$vp_end<MicroC_signal$OE_start])/250
MicroC_signal <- filter(MicroC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = MicroC_signal$vp_chr, 
                         ranges=IRanges(start=MicroC_signal$vp_start, end=MicroC_signal$vp_end))
CRISPRi_pos_non_ovrl <- filter(CRISPRi_pos, MicroC.loop==FALSE & overlaps.CTCF==FALSE)
CRISPRi_pos_non_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
CRISPRi_pos_non_ovrl_heatmap_ICE <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_pos_non_ovrl))){
  loop <- CRISPRi_pos_non_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- MicroC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal_n <- loop_signal[,c(9,7)]
  loop_signal_ICE <- loop_signal[,c(9,8)]
  colnames(loop_signal_n)[2] <- paste0("loop_",i)
  colnames(loop_signal_ICE)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_pos_non_ovrl_heatmap <- left_join(CRISPRi_pos_non_ovrl_heatmap, loop_signal_n)})
  suppressMessages({CRISPRi_pos_non_ovrl_heatmap_ICE <- left_join(CRISPRi_pos_non_ovrl_heatmap_ICE, loop_signal_ICE)})}
CRISPRi_pos_non_ovrl_heatmap[is.na(CRISPRi_pos_non_ovrl_heatmap)] <- 0
CRISPRi_pos_non_ovrl_heatmap_ICE[is.na(CRISPRi_pos_non_ovrl_heatmap_ICE)] <- 0
CRISPRi_pos_non_ovrl_heatmap <- as.data.frame(t(CRISPRi_pos_non_ovrl_heatmap)[2:ncol(CRISPRi_pos_non_ovrl_heatmap),])
CRISPRi_pos_non_ovrl_heatmap_ICE <- as.data.frame(t(CRISPRi_pos_non_ovrl_heatmap_ICE)[2:ncol(CRISPRi_pos_non_ovrl_heatmap_ICE),])
CRISPRi_pos_non_ovrl_heatmap <- arrange(CRISPRi_pos_non_ovrl_heatmap, rowMeans(CRISPRi_pos_non_ovrl_heatmap))
CRISPRi_pos_non_ovrl_heatmap_ICE <- arrange(CRISPRi_pos_non_ovrl_heatmap_ICE, rowMeans(CRISPRi_pos_non_ovrl_heatmap_ICE))
CRISPRi_pos_non_ovrl_heatmap_mod <- CRISPRi_pos_non_ovrl_heatmap
CRISPRi_pos_non_ovrl_heatmap_mod[CRISPRi_pos_non_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_MicroC_none_EP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_pos_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_pos_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# raw profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_non_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Micro-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_MicroC_none_EP_profile.pdf",device="pdf")

In [ ]:
# ICE profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=1000*colMeans(CRISPRi_pos_non_ovrl_heatmap_ICE)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Micro-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_MicroC_none_EP_profile_ICE.pdf",device="pdf")

# DHS-P pairs physically linked in Micro-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_neg$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_neg$target_gene.start, end=CRISPRi_neg$target_gene.start+1))
MicroC_signal <- read.csv("MicroC_output/MicroC.signal.df.txt", header = F, sep = "\t") %>% filter(V1!="")
colnames(MicroC_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum", "N_sum_ICE")
signal_ranges <- GRanges(seqnames = MicroC_signal$vp_chr, 
                         ranges=IRanges(start=MicroC_signal$vp_start, end=MicroC_signal$vp_end))
MicroC_signal <- MicroC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
MicroC_signal$dist_rank <- 0
MicroC_signal$dist_rank[MicroC_signal$vp_start>MicroC_signal$OE_start]<-(MicroC_signal$vp_start[MicroC_signal$vp_start>MicroC_signal$OE_start]- 
                                                            MicroC_signal$OE_start[MicroC_signal$vp_start>MicroC_signal$OE_start])/250
MicroC_signal$dist_rank[MicroC_signal$vp_end<MicroC_signal$OE_start]<-(MicroC_signal$OE_end[MicroC_signal$vp_end<MicroC_signal$OE_start]- 
                                                            MicroC_signal$vp_end[MicroC_signal$vp_end<MicroC_signal$OE_start])/250
MicroC_signal <- filter(MicroC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = MicroC_signal$vp_chr, 
                         ranges=IRanges(start=MicroC_signal$vp_start, end=MicroC_signal$vp_end))
CRISPRi_neg_ovrl <- filter(CRISPRi_neg, MicroC.loop==TRUE & overlaps.CTCF==FALSE)
CRISPRi_neg_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
CRISPRi_neg_ovrl_heatmap_ICE <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_neg_ovrl))){
  loop <- CRISPRi_neg_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- MicroC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal_n <- loop_signal[,c(9,7)]
  loop_signal_ICE <- loop_signal[,c(9,8)]
  colnames(loop_signal_n)[2] <- paste0("loop_",i)
  colnames(loop_signal_ICE)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_neg_ovrl_heatmap <- left_join(CRISPRi_neg_ovrl_heatmap, loop_signal_n)})
  suppressMessages({CRISPRi_neg_ovrl_heatmap_ICE <- left_join(CRISPRi_neg_ovrl_heatmap_ICE, loop_signal_ICE)})}
CRISPRi_neg_ovrl_heatmap[is.na(CRISPRi_neg_ovrl_heatmap)] <- 0
CRISPRi_neg_ovrl_heatmap_ICE[is.na(CRISPRi_neg_ovrl_heatmap_ICE)] <- 0
CRISPRi_neg_ovrl_heatmap <- as.data.frame(t(CRISPRi_neg_ovrl_heatmap)[2:ncol(CRISPRi_neg_ovrl_heatmap),])
CRISPRi_neg_ovrl_heatmap_ICE <- as.data.frame(t(CRISPRi_neg_ovrl_heatmap_ICE)[2:ncol(CRISPRi_neg_ovrl_heatmap_ICE),])
CRISPRi_neg_ovrl_heatmap <- arrange(CRISPRi_neg_ovrl_heatmap, rowMeans(CRISPRi_neg_ovrl_heatmap))
CRISPRi_neg_ovrl_heatmap_ICE <- arrange(CRISPRi_neg_ovrl_heatmap_ICE, rowMeans(CRISPRi_neg_ovrl_heatmap_ICE))
CRISPRi_neg_ovrl_heatmap_mod <- CRISPRi_neg_ovrl_heatmap
CRISPRi_neg_ovrl_heatmap_mod[CRISPRi_neg_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_MicroC_int_DHSP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_neg_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_neg_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# raw profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Micro-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_MicroC_int_DHSP_profile.pdf",device="pdf")

In [ ]:
# ICE profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=1000*colMeans(CRISPRi_neg_ovrl_heatmap_ICE)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Micro-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_MicroC_int_DHSP_profile_ICE.pdf",device="pdf")

# DHS-P pairs with no physical link in Micro-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_neg$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_neg$target_gene.start, end=CRISPRi_neg$target_gene.start+1))
MicroC_signal <- read.csv("MicroC_output/MicroC.signal.df.txt", header = F, sep = "\t") %>% filter(V1!="")
colnames(MicroC_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum", "N_sum_ICE")
signal_ranges <- GRanges(seqnames = MicroC_signal$vp_chr, 
                         ranges=IRanges(start=MicroC_signal$vp_start, end=MicroC_signal$vp_end))
MicroC_signal <- MicroC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
MicroC_signal$dist_rank <- 0
MicroC_signal$dist_rank[MicroC_signal$vp_start>MicroC_signal$OE_start]<-(MicroC_signal$vp_start[MicroC_signal$vp_start>MicroC_signal$OE_start]- 
                                                            MicroC_signal$OE_start[MicroC_signal$vp_start>MicroC_signal$OE_start])/250
MicroC_signal$dist_rank[MicroC_signal$vp_end<MicroC_signal$OE_start]<-(MicroC_signal$OE_end[MicroC_signal$vp_end<MicroC_signal$OE_start]- 
                                                            MicroC_signal$vp_end[MicroC_signal$vp_end<MicroC_signal$OE_start])/250
MicroC_signal <- filter(MicroC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = MicroC_signal$vp_chr, 
                         ranges=IRanges(start=MicroC_signal$vp_start, end=MicroC_signal$vp_end))
CRISPRi_neg_non_ovrl <- filter(CRISPRi_neg, MicroC.loop==FALSE & overlaps.CTCF==FALSE)
CRISPRi_neg_non_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
CRISPRi_neg_non_ovrl_heatmap_ICE <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_neg_non_ovrl))){
  loop <- CRISPRi_neg_non_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- MicroC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal_n <- loop_signal[,c(9,7)]
  loop_signal_ICE <- loop_signal[,c(9,8)]
  colnames(loop_signal_n)[2] <- paste0("loop_",i)
  colnames(loop_signal_ICE)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_neg_non_ovrl_heatmap <- left_join(CRISPRi_neg_non_ovrl_heatmap, loop_signal_n)})
  suppressMessages({CRISPRi_neg_non_ovrl_heatmap_ICE <- left_join(CRISPRi_neg_non_ovrl_heatmap_ICE, loop_signal_ICE)})}
CRISPRi_neg_non_ovrl_heatmap[is.na(CRISPRi_neg_non_ovrl_heatmap)] <- 0
CRISPRi_neg_non_ovrl_heatmap_ICE[is.na(CRISPRi_neg_non_ovrl_heatmap_ICE)] <- 0
CRISPRi_neg_non_ovrl_heatmap <- as.data.frame(t(CRISPRi_neg_non_ovrl_heatmap)[2:ncol(CRISPRi_neg_non_ovrl_heatmap),])
CRISPRi_neg_non_ovrl_heatmap_ICE <- as.data.frame(t(CRISPRi_neg_non_ovrl_heatmap_ICE)[2:ncol(CRISPRi_neg_non_ovrl_heatmap_ICE),])
CRISPRi_neg_non_ovrl_heatmap <- arrange(CRISPRi_neg_non_ovrl_heatmap, rowMeans(CRISPRi_neg_non_ovrl_heatmap))
CRISPRi_neg_non_ovrl_heatmap_ICE <- arrange(CRISPRi_neg_non_ovrl_heatmap_ICE, rowMeans(CRISPRi_neg_non_ovrl_heatmap_ICE))
CRISPRi_neg_non_ovrl_heatmap_mod <- CRISPRi_neg_non_ovrl_heatmap
CRISPRi_neg_non_ovrl_heatmap_mod[CRISPRi_neg_non_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_MicroC_none_DHSP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_neg_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_neg_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# raw profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_non_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Micro-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_MicroC_none_DHSP_profile.pdf",device="pdf")

In [ ]:
# ICE profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=1000*colMeans(CRISPRi_neg_non_ovrl_heatmap_ICE)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Micro-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_MicroC_none_DHSP_profile_ICE.pdf",device="pdf")

# E-P pairs physically linked in Hi-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_pos$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_pos$target_gene.start, end=CRISPRi_pos$target_gene.start+1))
HiC_signal <- read.csv("HiC_output/HiC.signal.df.txt", header = F, sep = "\t") %>% filter(V1!="")
colnames(HiC_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum", "N_sum_ICE")
signal_ranges <- GRanges(seqnames = HiC_signal$vp_chr, 
                         ranges=IRanges(start=HiC_signal$vp_start, end=HiC_signal$vp_end))
HiC_signal <- HiC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
HiC_signal$dist_rank <- 0
HiC_signal$dist_rank[HiC_signal$vp_start>HiC_signal$OE_start]<-(HiC_signal$vp_start[HiC_signal$vp_start>HiC_signal$OE_start]- 
                                                            HiC_signal$OE_start[HiC_signal$vp_start>HiC_signal$OE_start])/250
HiC_signal$dist_rank[HiC_signal$vp_end<HiC_signal$OE_start]<-(HiC_signal$OE_end[HiC_signal$vp_end<HiC_signal$OE_start]- 
                                                            HiC_signal$vp_end[HiC_signal$vp_end<HiC_signal$OE_start])/250
HiC_signal <- filter(HiC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = HiC_signal$vp_chr, 
                         ranges=IRanges(start=HiC_signal$vp_start, end=HiC_signal$vp_end))
CRISPRi_pos_ovrl <- filter(CRISPRi_pos, HiC.loop==TRUE & overlaps.CTCF==FALSE)
CRISPRi_pos_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
CRISPRi_pos_ovrl_heatmap_ICE <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_pos_ovrl))){
  loop <- CRISPRi_pos_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- HiC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal_n <- loop_signal[,c(9,7)]
  loop_signal_ICE <- loop_signal[,c(9,8)]
  colnames(loop_signal_n)[2] <- paste0("loop_",i)
  colnames(loop_signal_ICE)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_pos_ovrl_heatmap <- left_join(CRISPRi_pos_ovrl_heatmap, loop_signal_n)})
  suppressMessages({CRISPRi_pos_ovrl_heatmap_ICE <- left_join(CRISPRi_pos_ovrl_heatmap_ICE, loop_signal_ICE)})}
CRISPRi_pos_ovrl_heatmap[is.na(CRISPRi_pos_ovrl_heatmap)] <- 0
CRISPRi_pos_ovrl_heatmap_ICE[is.na(CRISPRi_pos_ovrl_heatmap_ICE)] <- 0
CRISPRi_pos_ovrl_heatmap <- as.data.frame(t(CRISPRi_pos_ovrl_heatmap)[2:ncol(CRISPRi_pos_ovrl_heatmap),])
CRISPRi_pos_ovrl_heatmap_ICE <- as.data.frame(t(CRISPRi_pos_ovrl_heatmap_ICE)[2:ncol(CRISPRi_pos_ovrl_heatmap_ICE),])
CRISPRi_pos_ovrl_heatmap <- arrange(CRISPRi_pos_ovrl_heatmap, rowMeans(CRISPRi_pos_ovrl_heatmap))
CRISPRi_pos_ovrl_heatmap_ICE <- arrange(CRISPRi_pos_ovrl_heatmap_ICE, rowMeans(CRISPRi_pos_ovrl_heatmap_ICE))
CRISPRi_pos_ovrl_heatmap_mod <- CRISPRi_pos_ovrl_heatmap
CRISPRi_pos_ovrl_heatmap_mod[CRISPRi_pos_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_HiC_int_EP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_pos_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_pos_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# raw profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Hi-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_HiC_int_EP_profile.pdf",device="pdf")

In [ ]:
# ICE profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_ovrl_heatmap_ICE)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Hi-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_HiC_int_EP_profile_ICE.pdf",device="pdf")

# E-P pairs with no physical link in Hi-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_pos$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_pos$target_gene.start, end=CRISPRi_pos$target_gene.start+1))
HiC_signal <- read.csv("HiC_output/HiC.signal.df.txt", header = F, sep = "\t") %>% filter(V1!="")
colnames(HiC_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum", "N_sum_ICE")
signal_ranges <- GRanges(seqnames = HiC_signal$vp_chr, 
                         ranges=IRanges(start=HiC_signal$vp_start, end=HiC_signal$vp_end))
HiC_signal <- HiC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
HiC_signal$dist_rank <- 0
HiC_signal$dist_rank[HiC_signal$vp_start>HiC_signal$OE_start]<-(HiC_signal$vp_start[HiC_signal$vp_start>HiC_signal$OE_start]- 
                                                            HiC_signal$OE_start[HiC_signal$vp_start>HiC_signal$OE_start])/250
HiC_signal$dist_rank[HiC_signal$vp_end<HiC_signal$OE_start]<-(HiC_signal$OE_end[HiC_signal$vp_end<HiC_signal$OE_start]- 
                                                            HiC_signal$vp_end[HiC_signal$vp_end<HiC_signal$OE_start])/250
HiC_signal <- filter(HiC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = HiC_signal$vp_chr, 
                         ranges=IRanges(start=HiC_signal$vp_start, end=HiC_signal$vp_end))
CRISPRi_pos_non_ovrl <- filter(CRISPRi_pos, HiC.loop==FALSE & overlaps.CTCF==FALSE)
CRISPRi_pos_non_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
CRISPRi_pos_non_ovrl_heatmap_ICE <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_pos_non_ovrl))){
  loop <- CRISPRi_pos_non_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- HiC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal_n <- loop_signal[,c(9,7)]
  loop_signal_ICE <- loop_signal[,c(9,8)]
  colnames(loop_signal_n)[2] <- paste0("loop_",i)
  colnames(loop_signal_ICE)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_pos_non_ovrl_heatmap <- left_join(CRISPRi_pos_non_ovrl_heatmap, loop_signal_n)})
  suppressMessages({CRISPRi_pos_non_ovrl_heatmap_ICE <- left_join(CRISPRi_pos_non_ovrl_heatmap_ICE, loop_signal_ICE)})}
CRISPRi_pos_non_ovrl_heatmap[is.na(CRISPRi_pos_non_ovrl_heatmap)] <- 0
CRISPRi_pos_non_ovrl_heatmap_ICE[is.na(CRISPRi_pos_non_ovrl_heatmap_ICE)] <- 0
CRISPRi_pos_non_ovrl_heatmap <- as.data.frame(t(CRISPRi_pos_non_ovrl_heatmap)[2:ncol(CRISPRi_pos_non_ovrl_heatmap),])
CRISPRi_pos_non_ovrl_heatmap_ICE <- as.data.frame(t(CRISPRi_pos_non_ovrl_heatmap_ICE)[2:ncol(CRISPRi_pos_non_ovrl_heatmap_ICE),])
CRISPRi_pos_non_ovrl_heatmap <- arrange(CRISPRi_pos_non_ovrl_heatmap, rowMeans(CRISPRi_pos_non_ovrl_heatmap))
CRISPRi_pos_non_ovrl_heatmap_ICE <- arrange(CRISPRi_pos_non_ovrl_heatmap_ICE, rowMeans(CRISPRi_pos_non_ovrl_heatmap_ICE))
CRISPRi_pos_non_ovrl_heatmap_mod <- CRISPRi_pos_non_ovrl_heatmap
CRISPRi_pos_non_ovrl_heatmap_mod[CRISPRi_pos_non_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_HiC_none_EP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_pos_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_pos_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# raw profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_non_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Hi-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_HiC_none_EP_profile.pdf",device="pdf")

In [ ]:
# ICE profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_pos_non_ovrl_heatmap_ICE)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Hi-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_HiC_none_EP_profile_ICE.pdf",device="pdf")

# DHS-P pairs physically linked in Hi-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_neg$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_neg$target_gene.start, end=CRISPRi_neg$target_gene.start+1))
HiC_signal <- read.csv("HiC_output/HiC.signal.df.txt", header = F, sep = "\t") %>% filter(V1!="")
colnames(HiC_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum", "N_sum_ICE")
signal_ranges <- GRanges(seqnames = HiC_signal$vp_chr, 
                         ranges=IRanges(start=HiC_signal$vp_start, end=HiC_signal$vp_end))
HiC_signal <- HiC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
HiC_signal$dist_rank <- 0
HiC_signal$dist_rank[HiC_signal$vp_start>HiC_signal$OE_start]<-(HiC_signal$vp_start[HiC_signal$vp_start>HiC_signal$OE_start]- 
                                                            HiC_signal$OE_start[HiC_signal$vp_start>HiC_signal$OE_start])/250
HiC_signal$dist_rank[HiC_signal$vp_end<HiC_signal$OE_start]<-(HiC_signal$OE_end[HiC_signal$vp_end<HiC_signal$OE_start]- 
                                                            HiC_signal$vp_end[HiC_signal$vp_end<HiC_signal$OE_start])/250
HiC_signal <- filter(HiC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = HiC_signal$vp_chr, 
                         ranges=IRanges(start=HiC_signal$vp_start, end=HiC_signal$vp_end))
CRISPRi_neg_ovrl <- filter(CRISPRi_neg, HiC.loop==TRUE & overlaps.CTCF==FALSE)
CRISPRi_neg_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
CRISPRi_neg_ovrl_heatmap_ICE <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_neg_ovrl))){
  loop <- CRISPRi_neg_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- HiC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal_n <- loop_signal[,c(9,7)]
  loop_signal_ICE <- loop_signal[,c(9,8)]
  colnames(loop_signal_n)[2] <- paste0("loop_",i)
  colnames(loop_signal_ICE)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_neg_ovrl_heatmap <- left_join(CRISPRi_neg_ovrl_heatmap, loop_signal_n)})
  suppressMessages({CRISPRi_neg_ovrl_heatmap_ICE <- left_join(CRISPRi_neg_ovrl_heatmap_ICE, loop_signal_ICE)})}
CRISPRi_neg_ovrl_heatmap[is.na(CRISPRi_neg_ovrl_heatmap)] <- 0
CRISPRi_neg_ovrl_heatmap_ICE[is.na(CRISPRi_neg_ovrl_heatmap_ICE)] <- 0
CRISPRi_neg_ovrl_heatmap <- as.data.frame(t(CRISPRi_neg_ovrl_heatmap)[2:ncol(CRISPRi_neg_ovrl_heatmap),])
CRISPRi_neg_ovrl_heatmap_ICE <- as.data.frame(t(CRISPRi_neg_ovrl_heatmap_ICE)[2:ncol(CRISPRi_neg_ovrl_heatmap_ICE),])
CRISPRi_neg_ovrl_heatmap <- arrange(CRISPRi_neg_ovrl_heatmap, rowMeans(CRISPRi_neg_ovrl_heatmap))
CRISPRi_neg_ovrl_heatmap_ICE <- arrange(CRISPRi_neg_ovrl_heatmap_ICE, rowMeans(CRISPRi_neg_ovrl_heatmap_ICE))
CRISPRi_neg_ovrl_heatmap_mod <- CRISPRi_neg_ovrl_heatmap
CRISPRi_neg_ovrl_heatmap_mod[CRISPRi_neg_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_HiC_int_DHSP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_neg_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_neg_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# raw profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Hi-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_HiC_int_DHSP_profile.pdf",device="pdf")

In [ ]:
# ICE profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_ovrl_heatmap_ICE)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Hi-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_HiC_int_DHSP_profile_ICE.pdf",device="pdf")

# DHS-P pairs with no physical link in Hi-C dataset

In [ ]:
rm(list=setdiff(ls(), c("CRISPRi_pos", "CRISPRi_neg")))
# heatmap
CRISPRi_ranges <- GRanges(seqnames = CRISPRi_neg$target_site.chr, 
                          ranges=IRanges(start=CRISPRi_neg$target_gene.start, end=CRISPRi_neg$target_gene.start+1))
HiC_signal <- read.csv("HiC_output/HiC.signal.df.txt", header = F, sep = "\t") %>% filter(V1!="")
colnames(HiC_signal) <- c("vp_chr","vp_start","vp_end","OE_chr","OE_start","OE_end","N_sum", "N_sum_ICE")
signal_ranges <- GRanges(seqnames = HiC_signal$vp_chr, 
                         ranges=IRanges(start=HiC_signal$vp_start, end=HiC_signal$vp_end))
HiC_signal <- HiC_signal[unique(as.data.frame(findOverlaps(signal_ranges, CRISPRi_ranges))$queryHits),]
HiC_signal$dist_rank <- 0
HiC_signal$dist_rank[HiC_signal$vp_start>HiC_signal$OE_start]<-(HiC_signal$vp_start[HiC_signal$vp_start>HiC_signal$OE_start]- 
                                                            HiC_signal$OE_start[HiC_signal$vp_start>HiC_signal$OE_start])/250
HiC_signal$dist_rank[HiC_signal$vp_end<HiC_signal$OE_start]<-(HiC_signal$OE_end[HiC_signal$vp_end<HiC_signal$OE_start]- 
                                                            HiC_signal$vp_end[HiC_signal$vp_end<HiC_signal$OE_start])/250
HiC_signal <- filter(HiC_signal, dist_rank>19)
signal_ranges <- GRanges(seqnames = HiC_signal$vp_chr, 
                         ranges=IRanges(start=HiC_signal$vp_start, end=HiC_signal$vp_end))
CRISPRi_neg_non_ovrl <- filter(CRISPRi_neg, HiC.loop==FALSE & overlaps.CTCF==FALSE)
CRISPRi_neg_non_ovrl_heatmap <- tibble(dist_rank=seq(-19,19))
CRISPRi_neg_non_ovrl_heatmap_ICE <- tibble(dist_rank=seq(-19,19))
for (i in seq(nrow(CRISPRi_neg_non_ovrl))){
  loop <- CRISPRi_neg_non_ovrl[i,]
  loop_ranges <- GRanges(seqnames = loop$target_site.chr, 
                         ranges=IRanges(start=loop$target_gene.start, end=loop$target_gene.start+1))
  loop_signal <- HiC_signal[as.data.frame(findOverlaps(signal_ranges, loop_ranges))$queryHits,]
  loop_signal[loop_signal$vp_start > loop_signal$OE_start,]$dist_rank <- -1 * loop_signal[loop_signal$vp_start > 
                                                                                loop_signal$OE_start,]$dist_rank
  if (loop$target_gene.start > loop$target_site.start) {loop_signal <- filter(loop_signal, dist_rank >
                            floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)-20 &
                            dist_rank<floor(((loop$target_site.start+loop$target_site.stop)/2 - vp_start)/250)+20)}
      else {loop_signal <- filter(loop_signal, dist_rank>ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)-20 & dist_rank<ceiling(((loop$target_site.start+
            loop$target_site.stop)/2 - vp_end)/250)+20)}
  if (loop$target_gene.start > loop$target_site.start) {loop_signal$dist_rank <- loop_signal$dist_rank -
     floor(((loop$target_site.start+loop$target_site.stop)/2 - loop_signal$vp_start)/250)} 
    else {loop_signal$dist_rank <- loop_signal$dist_rank - ceiling(((loop$target_site.start+
          loop$target_site.stop)/2 - loop_signal$vp_end)/250)}
  loop_signal_n <- loop_signal[,c(9,7)]
  loop_signal_ICE <- loop_signal[,c(9,8)]
  colnames(loop_signal_n)[2] <- paste0("loop_",i)
  colnames(loop_signal_ICE)[2] <- paste0("loop_",i)
  suppressMessages({CRISPRi_neg_non_ovrl_heatmap <- left_join(CRISPRi_neg_non_ovrl_heatmap, loop_signal_n)})
  suppressMessages({CRISPRi_neg_non_ovrl_heatmap_ICE <- left_join(CRISPRi_neg_non_ovrl_heatmap_ICE, loop_signal_ICE)})}
CRISPRi_neg_non_ovrl_heatmap[is.na(CRISPRi_neg_non_ovrl_heatmap)] <- 0
CRISPRi_neg_non_ovrl_heatmap_ICE[is.na(CRISPRi_neg_non_ovrl_heatmap_ICE)] <- 0
CRISPRi_neg_non_ovrl_heatmap <- as.data.frame(t(CRISPRi_neg_non_ovrl_heatmap)[2:ncol(CRISPRi_neg_non_ovrl_heatmap),])
CRISPRi_neg_non_ovrl_heatmap_ICE <- as.data.frame(t(CRISPRi_neg_non_ovrl_heatmap_ICE)[2:ncol(CRISPRi_neg_non_ovrl_heatmap_ICE),])
CRISPRi_neg_non_ovrl_heatmap <- arrange(CRISPRi_neg_non_ovrl_heatmap, rowMeans(CRISPRi_neg_non_ovrl_heatmap))
CRISPRi_neg_non_ovrl_heatmap_ICE <- arrange(CRISPRi_neg_non_ovrl_heatmap_ICE, rowMeans(CRISPRi_neg_non_ovrl_heatmap_ICE))
CRISPRi_neg_non_ovrl_heatmap_mod <- CRISPRi_neg_non_ovrl_heatmap
CRISPRi_neg_non_ovrl_heatmap_mod[CRISPRi_neg_non_ovrl_heatmap_mod>20] <- 20
pdf("Figures/Fig.S6/S6a_HiC_none_DHSP_heatmap.pdf")
heatmap(as.matrix(CRISPRi_neg_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)
dev.off()
heatmap(as.matrix(CRISPRi_neg_non_ovrl_heatmap_mod), Colv=NA, Rowv=NA, scale="none", labRow=FALSE, labCol=FALSE)

In [ ]:
# raw profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_non_ovrl_heatmap)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Hi-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_HiC_none_DHSP_profile.pdf",device="pdf")

In [ ]:
# ICE profile
ggplot()+geom_line(aes(x=seq(-19,19,1),y=colMeans(CRISPRi_neg_non_ovrl_heatmap_ICE)),size=2)+theme_classic(base_size = 20)+
xlim(-19,19) + ylim(0,17) + xlab("Distance from enhancer (250bp bins)")+ylab("Hi-C signal")

In [ ]:
ggsave("Figures/Fig.S6/S6a_HiC_none_DHSP_profile_ICE.pdf",device="pdf")

# Plotting precision-recall and ROC curves

In [ ]:
rm(list=ls())
# loading E-P and DHS-P data and combining them
positive <- read.csv("Summary_output_datasets/E-P_pairs.txt", header = T, sep = "\t")
negative <- read.csv("Summary_output_datasets/DHS-P_pairs.txt", header = T, sep = "\t")
combined <- rbind(positive, negative)
all_TSS <- read.csv("Auxiliary_data/RefSeqCurated.170308.bed.CollapsedGeneBounds.TSS.500bp.bed", header = F, sep = "\t")
expression <- read.csv("Auxiliary_data/ENCFF934YBO.tsv", header = T, sep = "\t")[,c(1,6)]
genes <- readGFF("Auxiliary_data/gencode.v19.annotation.gtf.gz")
genes <- filter(genes, type=="gene")[,c(9,13)] %>% distinct() 
expression <- left_join(expression,genes)
all_TSS <- left_join(all_TSS, expression[,c(2,3)], by=c("V4"="gene_name")) %>% filter(TPM>0.5)
output <- data.frame()
chromosomes <- unique(all_TSS$V1)
for (chromosome in chromosomes){
  combined_ss <- filter(combined, target_site.chr==chromosome)
  all_TSS_ss <- filter(all_TSS, V1==chromosome)
  combined_ranges <- GRanges(seqnames = combined_ss$target_site.chr, ranges=IRanges(start=combined_ss$target_site.start, end=combined_ss$target_site.stop, names = seq(1,nrow(combined_ss))))
  TSS_ranges <- GRanges(seqnames = all_TSS_ss$V1, ranges=IRanges(start=all_TSS_ss$V2, end=all_TSS_ss$V3,  names = all_TSS_ss$V4))
  combined_ss$closest_gene_k1 <- all_TSS_ss$V4[transpose(as.data.frame(do.call(cbind, nearestKNeighbors(combined_ranges, TSS_ranges, k=1))))$V1]==combined_ss$gene_short_name 
  combined_ss$closest_gene_k2 <- ((all_TSS_ss$V4[transpose(as.data.frame(do.call(cbind, nearestKNeighbors(combined_ranges, TSS_ranges, k=2))))$V1]==combined_ss$gene_short_name) | (all_TSS_ss$V4[transpose(as.data.frame(do.call(cbind, nearestKNeighbors(combined_ranges, TSS_ranges, k=2))))$V2]==combined_ss$gene_short_name))
  output <- bind_rows(output,combined_ss)
}

output$too_close_for_PLAC <- pmin(abs(output$target_gene.start - output$target_site.start), abs(output$target_gene.start - output$target_site.stop)) < 10000

sensitivity <- c(nrow(filter(output, Significant&MChIPC.loop))/nrow(filter(output, Significant)), # sensitivity mchipc
nrow(filter(output, Significant&MChIPC.sub05.loop))/nrow(filter(output, Significant)), # sensitivity 50% ss mchipc
nrow(filter(output, Significant&MChIPC.sub025.loop))/nrow(filter(output, Significant)), # sensitivity 25% ss mchipc
nrow(filter(output, Significant&MChIPC.sub01.loop))/nrow(filter(output, Significant)), # sensitivity 10% ss mchipc
nrow(filter(output, Significant&PLAC.loop&!(to_close_for_PLAC)))/nrow(filter(output, Significant&!(to_close_for_PLAC))), # sensitivity plac-seq
nrow(filter(output, Significant&MicroC.loop))/nrow(filter(output, Significant)), # sensitivity MicroC
nrow(filter(output, Significant&HiC.loop))/nrow(filter(output, Significant)), # sensitivity HiC
nrow(filter(output, Significant&closest_gene_k1))/nrow(filter(output, Significant)),# sensitivity the closest active promoter
nrow(filter(output, Significant&closest_gene_k2))/nrow(filter(output, Significant)))# sensitivity the 2 closest active promoters

precision <- c(nrow(filter(output, Significant&MChIPC.loop))/nrow(filter(output, MChIPC.loop)), # precision mchipc
nrow(filter(output, Significant&MChIPC.sub05.loop))/nrow(filter(output, MChIPC.sub05.loop)), # precision 50% ss mchipc
nrow(filter(output, Significant&MChIPC.sub025.loop))/nrow(filter(output, MChIPC.sub025.loop)), # precision 25% ss mchipc
nrow(filter(output, Significant&MChIPC.sub01.loop))/nrow(filter(output, MChIPC.sub01.loop)), # precision 10% ss mchipc
nrow(filter(output, Significant&PLAC.loop&!(to_close_for_PLAC)))/nrow(filter(output, PLAC.loop&!(to_close_for_PLAC))), # precision plac-seq
nrow(filter(output, Significant&MicroC.loop))/nrow(filter(output, MicroC.loop)), # precision MicroC
nrow(filter(output, Significant&HiC.loop))/nrow(filter(output, HiC.loop)), # precision HiC
nrow(filter(output, Significant&closest_gene_k1))/nrow(filter(output, closest_gene_k1)), # precision the closest active promoter
nrow(filter(output, Significant&closest_gene_k2))/nrow(filter(output, closest_gene_k2)))# precision the 2 closest active promoters

FPR <- c(nrow(filter(output, !(Significant)&MChIPC.loop))/nrow(filter(output, !(Significant))), # FPR mchipc
nrow(filter(output, !(Significant)&MChIPC.sub05.loop))/nrow(filter(output, !(Significant))), # FPR 50% ss mchipc
nrow(filter(output, !(Significant)&MChIPC.sub025.loop))/nrow(filter(output, !(Significant))), # FPR 25% ss mchipc
nrow(filter(output, !(Significant)&MChIPC.sub01.loop))/nrow(filter(output, !(Significant))), # FPR 10% ss mchipc
nrow(filter(output, !(Significant)&PLAC.loop&!(to_close_for_PLAC)))/nrow(filter(output, !(Significant)&!(to_close_for_PLAC))), # FPR plac-seq
nrow(filter(output, !(Significant)&MicroC.loop))/nrow(filter(output, !(Significant))), # FPR MicroC
nrow(filter(output, !(Significant)&HiC.loop))/nrow(filter(output, !(Significant))), # FPR HiC
nrow(filter(output, !(Significant)&closest_gene_k1))/nrow(filter(output, !(Significant))), # FPR the closest active promoter
nrow(filter(output, !(Significant)&closest_gene_k2))/nrow(filter(output, !(Significant))))# FPR the 2 closest active promoters

bm_data <- data.frame(sensitivity, precision, FPR) 
row.names(bm_data) <- c("MChIPC", "MChIPC.sub05","MChIPC.sub025", "MChIPC.sub01", "PLAC", "MicroC","HiC", "closest_k1", "closest_k2")

In [ ]:
pr_distance <- pr.curve(scores.class0=1/(abs(output$target_gene.start-output$target_site.start)), weights.class0=output$Significant, curve = TRUE)
pdf("Figures/Fig.S6/S6b_PR_curve.pdf")
plot.new()
grid(nx = NULL, ny = NULL, lty = 0.2, lwd = 1)
par(new = TRUE)
plot(pr_distance, color=1, auc.main = FALSE, main="", cex.lab=1.5, cex.axis=1.5, ylim=c(0,0.3))
points(bm_data$sensitivity, bm_data$precision, pch = 20)
text(bm_data$sensitivity, bm_data$precision, labels=rownames(bm_data), adj=c(0.5,-0.5), cex=1)
dev.off()
plot.new()
grid(nx = NULL, ny = NULL, lty = 0.2, lwd = 1)
par(new = TRUE)
plot(pr_distance, color=1, auc.main = FALSE, main="", cex.lab=1.5, cex.axis=1.5, ylim=c(0,0.3))
points(bm_data$sensitivity, bm_data$precision, pch = 20)
text(bm_data$sensitivity, bm_data$precision, labels=rownames(bm_data), adj=c(0.5,-0.5), cex=1)

In [ ]:
roc.distance <- roc.curve( scores.class0=1/(abs(output$target_gene.start-output$target_site.start)), weights.class0=output$Significant, curve = TRUE)
pdf("Figures/Fig.S6/S6b_ROC_curve.pdf")
plot.new()
grid(nx = NULL, ny = NULL, lty = 0.2, lwd = 1)
par(new = TRUE)
plot(roc.distance, color=1, auc.main = FALSE, main="", cex.lab=1.5, cex.axis=1.5, xlim=c(0,0.3))
points(bm_data$FPR, bm_data$sensitivity, pch = 20)
text(bm_data$FPR, bm_data$sensitivity, labels=rownames(bm_data), adj=c(-0.2,0.5), cex=1)
dev.off()
plot.new()
grid(nx = NULL, ny = NULL, lty = 0.2, lwd = 1)
par(new = TRUE)
plot(roc.distance, color=1, auc.main = FALSE, main="", cex.lab=1.5, cex.axis=1.5, xlim=c(0,0.3))
points(bm_data$FPR, bm_data$sensitivity, pch = 20)
text(bm_data$FPR, bm_data$sensitivity, labels=rownames(bm_data), adj=c(-0.2,0.5), cex=1)